In [1]:
import pandas as pd
import numpy as np
import os 
import re
import matplotlib.pyplot as plt
import scipy
# from scipy.stats.stats import pearsonr 
import os
from pathlib import Path
import math
from statsmodels.stats.multitest import multipletests
import statsmodels
from scipy.stats import ttest_rel, ttest_ind
from scipy import stats

In [2]:
normal =pd.read_csv('data/lusc-rsem-fpkm-tcga_paired.txt', sep="\t", header=0)
print(len(normal))
print(normal.columns)
print(normal.head(3))

19648
Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'TCGA-43-7657', 'TCGA-58-8386',
       'TCGA-22-5478', 'TCGA-22-5472', 'TCGA-43-5670', 'TCGA-60-2709',
       'TCGA-22-5489', 'TCGA-77-8007', 'TCGA-22-5471', 'TCGA-22-4609',
       'TCGA-22-5482', 'TCGA-56-8082', 'TCGA-22-5483', 'TCGA-56-8623',
       'TCGA-33-4587', 'TCGA-56-7579', 'TCGA-43-3394', 'TCGA-34-8454',
       'TCGA-77-7338', 'TCGA-43-6143', 'TCGA-43-6773', 'TCGA-51-4080',
       'TCGA-34-7107', 'TCGA-39-5040', 'TCGA-43-6771', 'TCGA-92-7340',
       'TCGA-77-7138', 'TCGA-77-7142', 'TCGA-56-7823', 'TCGA-22-5491',
       'TCGA-22-5481', 'TCGA-90-7767', 'TCGA-77-8008', 'TCGA-56-7730',
       'TCGA-33-6737', 'TCGA-56-7582', 'TCGA-56-7731', 'TCGA-77-7337',
       'TCGA-43-7658', 'TCGA-56-8201', 'TCGA-85-7710', 'TCGA-56-7580',
       'TCGA-43-6647', 'TCGA-90-6837', 'TCGA-56-8083', 'TCGA-51-4079',
       'TCGA-56-7222', 'TCGA-22-4593', 'TCGA-51-4081', 'TCGA-56-8309'],
      dtype='object')
  Hugo_Symbol  Entrez_Gene_Id  TCGA-43-7657  TC

In [3]:
cancer = pd.read_csv('data/lusc-rsem-fpkm-tcga-t_paired.txt', sep="\t", header=0)
print(len(cancer))
print(cancer.columns)
print(cancer.head(3))

19648
Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'TCGA-43-7657', 'TCGA-58-8386',
       'TCGA-22-5478', 'TCGA-22-5472', 'TCGA-43-5670', 'TCGA-60-2709',
       'TCGA-22-5489', 'TCGA-77-8007', 'TCGA-22-5471', 'TCGA-22-4609',
       'TCGA-22-5482', 'TCGA-56-8082', 'TCGA-22-5483', 'TCGA-56-8623',
       'TCGA-33-4587', 'TCGA-56-7579', 'TCGA-43-3394', 'TCGA-34-8454',
       'TCGA-77-7338', 'TCGA-43-6143', 'TCGA-43-6773', 'TCGA-51-4080',
       'TCGA-34-7107', 'TCGA-39-5040', 'TCGA-43-6771', 'TCGA-92-7340',
       'TCGA-77-7138', 'TCGA-77-7142', 'TCGA-56-7823', 'TCGA-22-5491',
       'TCGA-22-5481', 'TCGA-90-7767', 'TCGA-77-8008', 'TCGA-56-7730',
       'TCGA-33-6737', 'TCGA-56-7582', 'TCGA-56-7731', 'TCGA-77-7337',
       'TCGA-43-7658', 'TCGA-56-8201', 'TCGA-85-7710', 'TCGA-56-7580',
       'TCGA-43-6647', 'TCGA-90-6837', 'TCGA-56-8083', 'TCGA-51-4079',
       'TCGA-56-7222', 'TCGA-22-4593', 'TCGA-51-4081', 'TCGA-56-8309'],
      dtype='object')
  Hugo_Symbol  Entrez_Gene_Id  TCGA-43-7657  TC

In [4]:
def check(normal, cancer):
    if not (normal.columns == cancer.columns).all():
        print('Different Columns')
    if not (len(normal) == len(cancer)):
        print('Different length')
    else: 
        print('checked')
# print(normal.columns == cancer.columns)
# print(len(normal) == len(cancer))
check(normal, cancer)

checked


In [5]:
print((normal.columns == cancer.columns).all())

True


In [6]:
print(len(normal.columns))

52


In [7]:
normal_x = normal[((normal == 0).astype(int).sum(axis=1))<25]
cancer_x = cancer[((cancer == 0).astype(int).sum(axis=1))<25]
print(len(normal_x), len(cancer_x))

17626 17717


In [8]:
names = np.intersect1d(normal_x.Hugo_Symbol, cancer_x.Hugo_Symbol)

normal_x = normal_x[normal_x['Hugo_Symbol'].isin(names)]
cancer_x = cancer_x[cancer_x['Hugo_Symbol'].isin(names)]
normal_x.dropna()
cancer_x.dropna()
print(len(normal_x), len(cancer_x))

17275 17275


In [9]:
normal_x
cancer_x

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,HIST3H2A,92815,336.79,500.46,703.28,287.01,486.75,70.51,145.02,14.03,...,3.06,420.68,109.66,106.63,1233.75,172.65,303.44,228.13,251.48,23.59
1,LIN7B,64130,105.15,212.78,102.25,212.78,172.65,244.57,105.89,152.28,...,135.24,135.24,151.22,395.18,295.11,120.94,114.36,96.68,277.20,153.34
2,LXN,56925,848.22,236.21,271.48,759.08,61.25,620.67,329.84,599.49,...,688.78,204.07,438.59,503.95,3039.30,607.87,106.63,507.46,255.00,346.29
3,CNKSR2,22866,32.59,8.51,45.85,6.16,49.21,11.91,12.27,15.00,...,1.38,6.62,6.11,1.66,33.54,3.11,0.82,3.32,27.25,6.16
4,SCML1,6322,84.63,74.58,67.12,57.89,102.97,132.44,66.65,57.08,...,165.57,119.26,87.65,53.57,232.94,67.12,64.80,90.14,198.47,154.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19641,ZNF521,25925,37.32,76.71,59.97,84.04,14.24,202.66,435.55,59.55,...,22.59,33.78,148.09,106.63,297.17,22.26,25.72,106.63,71.00,37.05
19642,SPINT2,10653,56265.94,3901.01,16843.62,12244.81,37901.36,8479.89,9606.86,10959.30,...,18052.61,15392.14,20030.74,20593.91,9808.75,11189.60,10733.74,10014.87,6983.79,23169.48
19643,HAVCR2,84868,74.58,432.53,128.79,208.38,13.93,633.73,348.71,420.68,...,150.17,57.49,236.21,353.59,29.48,202.66,33.78,73.54,55.89,480.04
19646,CTD-2116N17.1,0,6.89,10.79,8.51,6.84,9.13,9.93,14.78,15.56,...,8.85,6.21,4.94,17.64,28.65,14.78,27.64,15.80,31.00,4.86


In [10]:
check(normal_x, cancer_x)

checked


In [11]:
np.array(normal_x.iloc[0][2:])

array([62.12, 130.6, 33.06, 35.5, 73.03, 60.39, 92.05, 66.65, 54.33,
       15.56, 55.49, 30.34, 14.45, 100.83, 131.51, 59.55, 87.03, 71.5,
       23.08, 48.87, 72.52, 115.97, 45.21, 74.58, 8.92, 27.44, 40.64,
       35.76, 59.13, 20.86, 37.05, 48.18, 51.71, 49.56, 72.01, 21.94,
       27.44, 35.0, 76.17, 28.86, 90.77, 59.55, 40.07, 22.92, 29.91,
       82.29, 4.7, 37.32, 43.63, 77.25], dtype=object)

# Normality

### Independent tests

In [12]:
dfs = [normal_x, cancer_x]
for df in dfs:
    df['normality'] = ''
    for i in range (len(df)):
        sample = np.array(df.iloc[i][2:-1])
        stat,pval = stats.shapiro(sample)
        if pval > 0.05 :
            df.at[i,'normality'] = 'ND'
        else: 
            df.at[i,'normality'] = 'NOT'
    df.head(3)
#     print('-----------')

In [13]:
counter = 0
x = ['normal', 'cancer']
for df in dfs:
    norm = len(df[df['normality'] == 'ND'])
    not_norm = len(df[df['normality'] == 'NOT'])
    print('dataframe {} has {} normally distrbuted genes and {} not normally distributed'.format( x[counter],norm, not_norm))
    counter +=1

dataframe normal has 6994 normally distrbuted genes and 10281 not normally distributed
dataframe cancer has 2028 normally distrbuted genes and 15247 not normally distributed


In [14]:
normal_norm = normal_x[normal_x['normality'] == 'ND']
normal_not = normal_x[normal_x['normality'] == 'NOT']
normal_norm.dropna()
normal_not.dropna()
print(len(normal_norm), len(normal_not))

cancer_norm = cancer_x[cancer_x['normality'] == 'ND']
cancer_not = cancer_x[cancer_x['normality'] == 'NOT']
cancer_norm.dropna()
cancer_not.dropna()
print(len(cancer_norm), len(cancer_not))

6994 10281
2028 15247


In [15]:
len(normal_norm.Hugo_Symbol.dropna())

6175

In [16]:
len(cancer_norm.Hugo_Symbol.dropna())

1772

In [17]:
len(normal_x.Hugo_Symbol)

19347

In [18]:
len(normal_x.Hugo_Symbol.dropna())

17275

In [19]:
normal_intersect = np.intersect1d(normal_norm.Hugo_Symbol.dropna(), cancer_norm.Hugo_Symbol.dropna())
not_intersect = np.setdiff1d(normal_x.Hugo_Symbol.dropna(),normal_intersect)
# main_list = np.setdiff1d(list_2,list_1)
# yields the elements in `list_2` that are NOT in `list_1`
# not_intersect = np.intersect1d(normal_not.Hugo_Symbol.dropna(), cancer_not.Hugo_Symbol.dropna())
print(len(normal_intersect), len(not_intersect))

1124 16151


In [20]:
normal_norm = normal_norm[normal_norm['Hugo_Symbol'].isin(normal_intersect)]
cancer_norm = cancer_norm[cancer_norm['Hugo_Symbol'].isin(normal_intersect)]
print(len(normal_norm),len(cancer_norm))

1124 1124


### Only 1124 are normally distributed, so on the whole independent dataset we'll use Wolcoxon

In [21]:
indep = [normal_x.dropna().copy(), cancer_x.dropna().copy()]
for df in indep:
    df.drop('Hugo_Symbol',axis=1, inplace=True)
    df.drop('Entrez_Gene_Id',axis=1, inplace=True)
    df.drop('normality',axis=1, inplace=True)
    print(df.columns)

Index(['TCGA-43-7657', 'TCGA-58-8386', 'TCGA-22-5478', 'TCGA-22-5472',
       'TCGA-43-5670', 'TCGA-60-2709', 'TCGA-22-5489', 'TCGA-77-8007',
       'TCGA-22-5471', 'TCGA-22-4609', 'TCGA-22-5482', 'TCGA-56-8082',
       'TCGA-22-5483', 'TCGA-56-8623', 'TCGA-33-4587', 'TCGA-56-7579',
       'TCGA-43-3394', 'TCGA-34-8454', 'TCGA-77-7338', 'TCGA-43-6143',
       'TCGA-43-6773', 'TCGA-51-4080', 'TCGA-34-7107', 'TCGA-39-5040',
       'TCGA-43-6771', 'TCGA-92-7340', 'TCGA-77-7138', 'TCGA-77-7142',
       'TCGA-56-7823', 'TCGA-22-5491', 'TCGA-22-5481', 'TCGA-90-7767',
       'TCGA-77-8008', 'TCGA-56-7730', 'TCGA-33-6737', 'TCGA-56-7582',
       'TCGA-56-7731', 'TCGA-77-7337', 'TCGA-43-7658', 'TCGA-56-8201',
       'TCGA-85-7710', 'TCGA-56-7580', 'TCGA-43-6647', 'TCGA-90-6837',
       'TCGA-56-8083', 'TCGA-51-4079', 'TCGA-56-7222', 'TCGA-22-4593',
       'TCGA-51-4081', 'TCGA-56-8309'],
      dtype='object')
Index(['TCGA-43-7657', 'TCGA-58-8386', 'TCGA-22-5478', 'TCGA-22-5472',
       'TCGA-43

In [22]:
indep[0]

,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,TCGA-22-5471,TCGA-22-4609,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,62.12,130.60,33.06,35.50,73.03,60.39,92.05,66.65,54.33,15.56,...,90.77,59.55,40.07,22.92,29.91,82.29,4.70,37.32,43.63,77.25
1,185.11,283.05,119.26,169.07,165.57,161.02,131.51,198.47,175.07,147.06,...,185.11,119.26,102.97,123.50,264.03,194.36,166.73,105.15,185.11,356.05
2,909.17,819.30,412.00,743.43,1340.84,607.87,1709.26,1709.26,603.67,555.41,...,813.63,2400.97,543.96,2193.99,540.19,521.76,253.23,764.36,518.15,878.17
3,41.81,18.29,40.93,67.12,54.72,29.27,20.26,23.76,28.04,39.22,...,34.51,70.01,57.49,57.89,67.12,34.51,22.10,31.90,28.24,49.91
4,133.36,214.27,108.14,109.66,190.34,211.31,96.01,208.38,120.10,239.52,...,251.48,209.84,120.10,109.66,155.50,162.14,277.20,86.43,164.42,155.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19641,215.77,148.09,83.45,232.94,167.90,80.01,161.02,104.42,81.14,113.56,...,145.02,202.66,111.99,371.22,154.42,289.02,32.59,191.67,204.07,88.26
19642,6792.79,5441.30,5831.91,5329.30,5711.87,6164.49,8134.41,6516.03,8598.28,5633.22,...,3955.48,4869.99,6935.54,4836.35,7230.10,6250.56,6840.04,5292.48,3928.15,7130.55
19643,423.61,529.06,660.68,620.67,518.15,848.22,366.09,1073.91,363.56,366.09,...,559.28,579.04,420.68,366.09,334.46,258.57,329.84,1030.12,665.29,633.73
19646,0.00,0.00,0.99,3.59,2.23,3.35,0.00,7.34,1.41,0.00,...,6.94,1.00,0.00,4.58,0.96,3.06,1.83,1.77,3.63,2.56


In [ ]:
stat, p = stats.wilcoxon(indep[0], indep[1], axis=1, alternative='less')
# indep_wilcoxon = pd.DataFrame([normal_x.dropna().Hugo_Symbol, normal_x.dropna().Entrez_Gene_Id, normal_x.dropna().normality, p, stat])
indep_wilcoxon = pd.DataFrame(list(zip(normal_x.dropna().Hugo_Symbol, normal_x.dropna().Entrez_Gene_Id, normal_x.dropna().normality, p, stat)))
indep_wilcoxon.columns =['Hugo_Symbol', 'Entrez_Gene_Id', 'normality', 'p', 'stat']
indep_wilcoxon['expression'] = np.where(indep_wilcoxon['p']>0.05, 'not', 'expressed')
print(len(indep_wilcoxon))
indep_wilcoxon.head(10)

In [ ]:
indep[0].dropna(axis=1)

In [ ]:
p1

In [ ]:
stat, p = stats.wilcoxon(indep[0], indep[1], axis=1, alternative='less')
# print('stat=%.3f, p=%.3f' % (stat, p))
# L=[]
# for i in range (0,len(p)):
#     if p[i] > 0.05 :
#         L.append("ne")
#     else:
#         L.append("e")


In [ ]:
print(p)

In [ ]:
normal_not = normal_not[normal_not['Hugo_Symbol'].isin(not_intersect)]
cancer_not = cancer_not[cancer_not['Hugo_Symbol'].isin(not_intersect)]
print(len(normal_not),len(cancer_not))

In [ ]:
normal_norm.head(10)

In [ ]:
cancer_norm.head(10)

In [ ]:
normal_not.head(10)

In [ ]:
cancer_not.head(10)

In [ ]:
dfss = [normal_norm, cancer_norm, normal_not, cancer_not]
for df in dfss:
    df.drop('Hugo_Symbol')
    df.drop('Entrez_Gene_Id')
    df.drop('paired')